## This explores the sizes of objects



#### Standard inputs

In [ ]:
import os
import tables_io
import numpy as np
import matplotlib.pyplot as plt
import pyccl as ccl

Change this to be the correct location

In [ ]:
HOME = os.environ['HOME']
pz_dir = f'{HOME}/macss'

#### Set up the cosmology & the evaluation grids

In [ ]:
cosmo = ccl.Cosmology(
    Omega_c=0.25, Omega_b=0.05,
    h=0.7, n_s=0.95, sigma8=0.8,
    transfer_function='bbks'
)

z_grid = np.linspace(0., 6., 301)[1:]
a_grid = (1/(1+z_grid))
D_a = cosmo.angular_diameter_distance(a_grid)
rad_to_asec = 360*60*60/(2*np.pi) 
ang_scale = D_a * 1000/ rad_to_asec
dist_mod = cosmo.distance_modulus(a_grid)

#### Plot the apparent magntidue of the MW-like object (abs mag -20)

In [ ]:
_ = plt.plot(z_grid, dist_mod-20)
_ = plt.ylabel("Apparent Magntiude of MW like object")
_ = plt.xlabel("Redshift")

#### Plot the angular scale as a function of redshift

In [ ]:
_ = plt.plot(z_grid, ang_scale)
_ = plt.ylabel("Angular Scale [kpc/arcsec]")
_ = plt.xlabel("Redshift")

#### Plot the locus for MW-like and significantly brighter galaxy

In [ ]:
_ = plt.plot(dist_mod-20, 5/ang_scale, label="MW-like")
_ = plt.plot(dist_mod-22.5, 10/ang_scale, label="10x MW")
_ = plt.xlabel('Brightness [mag]')
_ = plt.ylabel('Size ["]')
_ = plt.legend()

#### Now read some DP1 data for comparision

In [ ]:
t = tables_io.read(f'{pz_dir}/data/object.hdf5')

#### Extract some useful quantities relating to the galaxy shape and size

In [ ]:
def r_95_of_n(n):
    return 1.25 + 1.125*n

In [ ]:
blendedness = t['i_blendedness']
blendedness = np.where(np.isnan(blendedness), 0., blendedness).clip(0., 1.)
mask  = np.bitwise_and(
    np.bitwise_and(
        t['i_extendedness'] == 1,
        t['g_psfFlux'] / t['g_psfFluxErr'] > 10,
    ),
    np.bitwise_and(
        t['i_psfFlux'] / t['i_psfFluxErr'] > 10,
        t['r_psfFlux'] / t['r_psfFluxErr'] > 10,
    ),
)
mask_unblend = np.bitwise_and(mask, blendedness<0.1)
mask_blend = np.bitwise_and(mask, blendedness>0.1)
blend = t['detect_fromBlend']
psf_flux = t['i_psfFlux']
cModel_flux = t['i_cModelFlux']
cModel_flux = t['i_sersicFlux']
shape_xx = t['i_ixx']
shape_yy = t['i_iyy']
shape_xy = t['i_ixy']
psf_xx = t['i_ixxDebiasedPSF']
psf_yy = t['i_iyyDebiasedPSF']
psf_xy = t['i_ixyDebiasedPSF']
shape_det = shape_xx*shape_yy - shape_xy*shape_xy
shape_trace = shape_xx + shape_yy
shape_area = np.sqrt(shape_det)
psf_det = psf_xx*psf_yy - psf_xy*psf_xy
psf_trace = psf_xx + psf_yy
sersic_x = t['sersic_reff_x']
sersic_y = t['sersic_reff_y']
sersic_rho = t['sersic_rho']
sersic_index = t['sersic_index']
sersic_factor = r_95_of_n(sersic_index)
sersic_xx = sersic_x*sersic_x
sersic_yy = sersic_y*sersic_y
sersic_trace = sersic_xx + sersic_yy
sersic_xy = sersic_x*sersic_y*sersic_rho
sersic_det = sersic_xx*sersic_yy - sersic_xy*sersic_xy
sqrt_sersic_trace = np.sqrt(sersic_trace)
sqrt_sersic_det = np.sqrt(sersic_det)
bdReB = t['i_bdReB']
bdReD = t['i_bdReD']
kron_rad = t['i_kronRad']
psf_mag = -2.5 * np.log10(t['i_psfFlux']) + 31.4
cModel_mag = -2.5 * np.log10(t['i_cModelFlux']) + 31.4
sersic_mag = -2.5 * np.log10(t['i_sersicFlux']) + 31.4

#### Let's plot some of these quantities

Here is a plot of the "trace" size for the galaxy moments and for the point-spread function.

Here is a drawing that shows a geometrical interpretation of the "Trace" size: https://docs.google.com/presentation/d/1XOIBx4M2QjBeErpwHVMjb4eMyaq5KVgMtT1s2Fp-kjI/edit?usp=sharing

In [ ]:
_ = plt.hist(np.sqrt(shape_trace[mask])*0.2, np.linspace(0, 2, 201), label='Moment')
_ = plt.hist(np.sqrt(psf_trace[mask])*0.2, np.linspace(0, 2, 201), label='PSF')
_ = plt.legend()
_ = plt.xlabel(r'$T^{1/2}$ ["]')

#### Here is a plot that shows the trace size for the Sersic fits, which have the point-spread function deconvolved.

In [ ]:
_ = plt.hist(np.sqrt(sersic_trace[mask_blend])*0.2, bins=np.linspace(0, 5, 201), label='all')
_ = plt.yscale('log')

#### Here we compare the sersic trace to the moment trace

We see that Sersic trace is typically smaller, as it has the PSF removed.

In [ ]:
_ = plt.hist2d(sersic_trace, shape_trace, bins=(np.linspace(0, 40, 201), np.linspace(0, 40., 201)), norm='log')
_ = plt.xlim(0, 40)
_ = plt.ylim(0, 40)
_ = plt.xlabel(r'$T_{\rm sersic}$')
_ = plt.ylabel(r'$T_{\rm moment}$')

#### Here we compare subtracting the PSF Trace from the moment trace with the Sersic trace and show that they are quite correlated

In [ ]:
_ = plt.hist2d(np.sqrt(shape_trace[mask]-psf_trace[mask])*0.2, np.sqrt(sersic_trace[mask])*0.2, bins=(np.linspace(0, 2, 201), np.linspace(0, 2., 201)), norm='log')
_ = plt.xlabel(r'$(T_{\rm moment} - T_{\rm psf})^{1/2}$ ["]')
_ = plt.ylabel(r'$(T_{\rm sersic})^{1/2}$ ["]')

### The next couple of plots show the size versus the magnitude of the galaxies in our sample

The first plot is for the Sersic trace, which is about equal to the half-light size of the galaxy, 
the curves show how galaxies similar to the Milky Way might show up in this plot.

The second plot accounts for the Sersic profile and shows the 90% light radius.

In [ ]:
_ = plt.hist2d(cModel_mag[mask], np.sqrt(sersic_trace[mask])*0.2, bins=(np.linspace(16, 26, 201), np.linspace(0, 4, 201)), norm='log')
_ = plt.xlabel(r'$i_{\rm cModel} [mag]$')
_ = plt.ylabel(r'$(T_{\rm sersic})^{1/2}$ ["]')
_ = plt.plot(dist_mod-20, 5./ang_scale, label="-20 [mag], 5kpc (MW-like)")
_ = plt.plot(dist_mod-22.5, 10./ang_scale, label="-22.5 [mag], 10kpc")
_ = plt.plot(dist_mod-22.5, 5./ang_scale, label="-22.5 [mag], 5kpc")
_ = plt.ylim(0, 5.)
_ = plt.legend()

In [ ]:
_ = plt.hist2d(cModel_mag[mask_unblend], sersic_factor[mask_unblend]*np.sqrt(sersic_trace[mask_unblend])*0.2, bins=(np.linspace(16, 26, 201), np.linspace(0, 10, 201)), norm='log')
_ = plt.xlabel(r'$i_{\rm cModel} [mag]$')
_ = plt.ylabel(r'$\frac{r_{\rm 90}}{r_{\rm 50}}(T_{\rm sersic})^{1/2}$ ["]')
_ = plt.plot(dist_mod-20, 10./ang_scale, label="-20 [mag], 10kpc (MW-like)")
_ = plt.plot(dist_mod-22.5, 10./ang_scale, label="-22.5 [mag], 10kpc")
_ = plt.plot(dist_mod-22.5, 5./ang_scale, label="-22.5 [mag], 5kpc")
_ = plt.ylim(0, 10.)
_ = plt.legend()